
# EEG Data Analysis Assignment


In this assignment you will analyze the EEG data that you collected yourself. There are different approaches to preprocessing. Here we will model our pipeline after this recent paper (check Methods): https://pmc.ncbi.nlm.nih.gov/articles/PMC10659264/pdf/nihpp-2023.11.07.566051v2.pdf.

## Part 1: Put all data in BIDS format

In [2]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from mne_bids import BIDSPath, write_raw_bids, read_raw_bids

# Define directories:
project_dir = os.getcwd()
print(project_dir)
raw_dir = os.path.join(project_dir, 'raw')
print(raw_dir)
bids_dir = os.path.join(project_dir, 'bids')

# Define event IDs:
event_id = {
    'A/standard': 1,
    'A/oddball':  2,
    'B/standard': 3,
    'B/oddball':  4,
}

# subject ID
subjects = ["01", '02', '03', '04', '05','06']

C:\Users\Gebruiker
C:\Users\Gebruiker\raw


🔥 Please loop over filenames and put all raw data in BIDS format (look up what this means). You can use the functions ```BIDSPath``` and ```write_raw_bids```. We need to take care of a couple of things.

First, after loading the raw EEG data, we need to change channels 'M1', 'M2', 'EXG7' and 'EXG8' to type 'misc', and channels 'UP', 'DOWN', 'LEFT' and 'RIGH' to type 'eog'. We also need to set the right montage ('biosemi64').

Second, observe that ```write_raw_bids``` takes the "events" and "event_id" parameters. During EEG acquisition we always sent the same trigger (2) at every sound presentation. However, we would like to include information about state and stimulus, as per the dictionary called "event_id" (see previous cell). Please make sure to: (i) find all events in the eeg file, (ii) read information about state and stimulus ID in the corresponding "*_events.tsv" file, (iii) update the events according to "event_id", (iv) pass the "events" and "event_id" in ```write_raw_bids```.

Third, look at all the json sidecar files, and check whether all information is correct. 


In [ ]:
def rawtobids(raw_dir, bids_dir, subjects, event_id):
    """
    Converteer alle .bdf bestanden in raw_dir naar BIDS in bids_dir.
    Voor elk bestand:
      - zet opgegeven kanaaltypes (misc, eog)
      - zet montage biosemi64
      - vind STIM-events in raw
      - lees corresponderende *_events.tsv (zelfde basename + '_events.tsv')
      - filter phase == "2" en map (state, frequency) naar de juiste event_id
      - zet de derde kolom van de events-array naar die nieuwe codes (alleen voor de fase2-triggers)
      - schrijf naar BIDS met write_raw_bids(..., events_data=events_array, event_id=event_id)
    """
    bdf_files = sorted(glob.glob(os.path.join(raw_dir, "*.bdf")))
    '''
    print("Gevonden .bdf bestanden:", bdf_files)
    '''
    print("aantal .bdf bestanden:", len(bdf_files))

    # Loop over alle bdf bestanden
    for i, bdf_file in enumerate(bdf_files):
        '''
        print("\nVerwerken:", bdf_file)
        '''
        # Lees raw
        raw = mne.io.read_raw_bdf(bdf_file, preload=True)

        # Types omzetten op basis van kanaal
        misc = ['M1', 'M2', 'EXG7', 'EXG8']
        raw.set_channel_types({ch: 'misc' for ch in misc if ch in raw.ch_names})
        eog = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        raw.set_channel_types({ch: 'eog' for ch in eog if ch in raw.ch_names})

        # montage biosemi64
        raw.set_montage('biosemi64', on_missing='ignore')

        # Zoek events.tsv bestand corresponderend met bdf_file
        prefix = bdf_file.split('_eeg.bdf')[0] + "_"    # files heten 1_2_eeg.bdf, prefix = "1_2_"
        events_tsv_path = prefix + "*_events.tsv"       # path naar 1_2_*kan alles zijn*_events.tsv  
        if not os.path.exists(events_tsv_path): # als file niet bestaat
            print(f"  Waarschuwing: events file niet gevonden: {events_tsv_path}. Schrijf BIDS zonder aangepaste events.")
            events_array_to_write = None
        else: # lees de file in en filter: alleen rows waar phase == "2"
            df = pd.read_csv(events_tsv_path, sep="\t", dtype=str)
            filtered = df[df["phase"].str.strip() == "2"].copy()
            '''
            print(f"  Aantal events in {os.path.basename(events_tsv_path)} met phase==2: {len(filtered)}")
            '''

            # Vind STIM-events in raw bdf file
            new_event_codes = []
            if not filtered.empty:
                # zet veilig om naar floats/integers waar nodig
                # (we volgen hetzelfde logic als in jouw oorspronkelijke code)
                for st_str, fq_str in zip(filtered["state"], filtered["frequency"]):
                    try:
                        st = float(str(st_str).strip())
                    except:
                        st = np.nan
                    try:
                        fq = float(str(fq_str).strip())
                    except:
                        fq = np.nan

                    if st == 1:  # reeks A
                        if fq == 2000:
                            new_event_codes.append(event_id["A/standard"])
                        else:
                            new_event_codes.append(event_id["A/oddball"])
                    elif st == -1:  # reeks B
                        if fq == 1000:
                            new_event_codes.append(event_id["B/standard"])
                        else:
                            new_event_codes.append(event_id["B/oddball"])
                    else:
                        new_event_codes.append(0)  # ongedefinieerd -> code 0 (volgt jouw eerdere aanpak)

            # -- combineer de gevonden stim_events met de nieuw gemapte codes --
            n_triggers = len(stim_events)
            n_filtered = len(new_event_codes)

            if n_triggers == 0:
                print("  Geen stim-events gevonden in raw; geen events meegeven aan write_raw_bids.")
                events_array_to_write = None
            else:
                if n_triggers != n_filtered:
                    print(f"  Let op: aantal gevonden stim-events ({n_triggers}) ≠ aantal gefilterde phase2-rows ({n_filtered}).")
                    print("  Ik zal de arrays afstemmen op de minimale lengte (geen fouten raise).")
                n_use = min(n_triggers, n_filtered)
                if n_use == 0:
                    events_array_to_write = None
                else:
                    events_array_to_write = stim_events[:n_use].copy()
                    events_array_to_write[:, 2] = np.array(new_event_codes[:n_use], dtype=int)
                    '''
                    print(f"  Events array klaargezet met {n_use} events (kolom 3 aangepast).")
                    '''

        # BIDSPath maken (subject index i -> subjects[i])
        bids_path = BIDSPath(subject=subjects[i],
                             root=bids_dir,
                             datatype='eeg',
                             extension='.bdf',
                             suffix='eeg',
                             task='experiment')

        # Schrijf naar BIDS
        if events_array_to_write is None:   # als events ontbreken alleen bdf schrijven
            write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')
            '''
            print(f"  {os.path.basename(bdf_file)} geschreven naar BIDS (zonder custom events).")
            '''
        else:
            write_raw_bids(raw,
                           bids_path=bids_path,
                           events_data=events_array_to_write,
                           event_id=event_id,
                           overwrite=True,
                           allow_preload=True,
                           format='EDF')
            print(f"  {os.path.basename(bdf_file)} geschreven naar BIDS.")

    print("\nKlaar met alle bestanden inbids zetten.")

# run functie
rawtobids(raw_dir=raw_dir, bids_dir=bids_dir, subjects=subjects, event_id=event_id)


aantal .bdf bestanden: 3
Extracting EDF parameters from C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\raw\1_2_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...


Reading 0 ... 479231  =      0.000 ...   467.999 secs...
  Waarschuwing: events file niet gevonden: C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\raw\1_2_*_events.tsv. Schrijf BIDS zonder aangepaste events.
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\participants.tsv'...
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\participants.json'...
Writing 'C:/Users/Gebruiker/Desktop/Code Data Science/Code-Data-Science/bids/sub-01/eeg/sub-01_space-CapTrak_electrodes.tsv'...
Writing 'C:/Users/Gebruiker/Desktop/Code Data Science/Code-Data-Science/bids/sub-01/eeg/sub-01_space-CapTrak_coordsystem.json'...
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\dataset_description.json'...
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\sub-01\eeg\sub-01_task-experiment_eeg.json'...
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\sub-01\eeg\sub

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\3956862619.py:27: RuntimeWarning: The unit for channel(s) EXG7, EXG8, M1, M2 has changed from V to NA.
  raw.set_channel_types({ch: 'misc' for ch in misc if ch in raw.ch_names})
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\3956862619.py:106: RuntimeWarning: No events found or provided. Please add annotations to the raw data, or provide the events and event_id parameters. For resting state data, BIDS recommends naming the task using labels beginning with "rest".
  write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\3956862619.py:106: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')


Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\sub-01\sub-01_scans.tsv'...
Wrote C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\sub-01\sub-01_scans.tsv entry with eeg\sub-01_task-experiment_eeg.edf.
Extracting EDF parameters from C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\raw\1_4_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 474111  =      0.000 ...   462.999 secs...
  Waarschuwing: events file niet gevonden: C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\raw\1_4_*_events.tsv. Schrijf BIDS zonder aangepaste events.
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\participants.tsv'...
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\participants.json'...
Writing 'C:/Users/Gebruiker/Desktop/Code Data Science/Code-Data-Science/bids/sub-02/eeg/sub-02_space-CapTrak_electrodes.tsv'...
Writi

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\3956862619.py:27: RuntimeWarning: The unit for channel(s) EXG7, EXG8, M1, M2 has changed from V to NA.
  raw.set_channel_types({ch: 'misc' for ch in misc if ch in raw.ch_names})
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\3956862619.py:106: RuntimeWarning: No events found or provided. Please add annotations to the raw data, or provide the events and event_id parameters. For resting state data, BIDS recommends naming the task using labels beginning with "rest".
  write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\3956862619.py:106: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')


Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\sub-02\sub-02_scans.tsv'...
Wrote C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\sub-02\sub-02_scans.tsv entry with eeg\sub-02_task-experiment_eeg.edf.
Extracting EDF parameters from C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\raw\sub-01.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 479231  =      0.000 ...   467.999 secs...
  Waarschuwing: events file niet gevonden: C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\raw\sub-01.bdf_*_events.tsv. Schrijf BIDS zonder aangepaste events.
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\participants.tsv'...
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\participants.json'...
Writing 'C:/Users/Gebruiker/Desktop/Code Data Science/Code-Data-Science/bids/sub-03/eeg/sub-03_space-CapTrak_electrodes.tsv'...

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\3956862619.py:27: RuntimeWarning: The unit for channel(s) EXG7, EXG8, M1, M2 has changed from V to NA.
  raw.set_channel_types({ch: 'misc' for ch in misc if ch in raw.ch_names})
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\3956862619.py:106: RuntimeWarning: No events found or provided. Please add annotations to the raw data, or provide the events and event_id parameters. For resting state data, BIDS recommends naming the task using labels beginning with "rest".
  write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\3956862619.py:106: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')


Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\sub-03\sub-03_scans.tsv'...
Wrote C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\sub-03\sub-03_scans.tsv entry with eeg\sub-03_task-experiment_eeg.edf.

Klaar met alle bestanden inbids zetten.


In [ ]:
def rawtobids(raw_dir, bids_dir, subjects, event_id):
    """
    Converteer alle .bdf bestanden in raw_dir naar BIDS in bids_dir.
    Voor elk bestand:
      - zet opgegeven kanaaltypes (misc, eog)
      - zet montage biosemi64
      - vind STIM-events in raw
      - lees corresponderende *_events.tsv (zelfde basename + '_events.tsv')
      - filter phase == "2" en map (state, frequency) naar de juiste event_id
      - zet de derde kolom van de events-array naar die nieuwe codes (alleen voor de fase2-triggers)
      - schrijf naar BIDS met write_raw_bids(..., events_data=events_array, event_id=event_id)
    """
    bdf_files = sorted(glob.glob(os.path.join(raw_dir, "*.bdf")))
    #print("Gevonden .bdf bestanden:", bdf_files)
    print("Gevonden .bdf bestanden:", len(bdf_files))

    for i, bdf_file in enumerate(bdf_files):
        #print("\nVerwerken:", bdf_file)
        # lees raw bdf files in
        raw = mne.io.read_raw_bdf(bdf_file, preload=True)

        # Types omzetten op basis van kanaal
        misc = ['M1', 'M2', 'EXG7', 'EXG8']
        raw.set_channel_types({ch: 'misc' for ch in misc if ch in raw.ch_names})
        eog = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        raw.set_channel_types({ch: 'eog' for ch in eog if ch in raw.ch_names})

        # montage biosemi64
        raw.set_montage('biosemi64', on_missing='ignore')

        # Zoek events.tsv bestand corresponderend met bdf_file
        prefix = bdf_file.split('_eeg.bdf')[0] + "_"    # files heten 1_2_eeg.bdf, prefix = "1_2_"
        events_tsv_path = prefix + "*_events.tsv"       # path naar 1_2_*kan alles zijn*_events.tsv  
        if not os.path.exists(events_tsv_path): # als file niet bestaat
            print(f"  Waarschuwing: events file niet gevonden: {events_tsv_path}. Schrijf BIDS zonder aangepaste events.")
            events_array_to_write = None
        else: # lees de file in en filter: alleen rows waar phase == "2"
            df = pd.read_csv(events_tsv_path, sep="\t", dtype=str)
            filtered = df[df["phase"].str.strip() == "2"].copy()
            print(f"  Aantal events in {os.path.basename(events_tsv_path)} met phase==2: {len(filtered)}")

            # Maak een nieuwe lijst voor de aangepaste event codes zoals in de dictionary
            new_event_codes = []

             # Loop door de rijen van de DataFrame en wijs de juiste event code toe
            if st == 1:  # reeks A
                if fq == 2000:
                    new_event_codes.append(event_id["A/standard"])
                else:
                    new_event_codes.append(event_id["A/oddball"])
            elif st == -1:  # reeks B
                if fq == 1000:
                    new_event_codes.append(event_id["B/standard"])
                else:
                    new_event_codes.append(event_id["B/oddball"])
            else:
                new_event_codes.append(0)  # ongedefinieerd -> code 0 (volgt jouw eerdere aanpak)

            n_triggers = len(stim_events)
            n_filtered = len(new_event_codes)

            # code om errors te voorkomen: (dit was een issue een paar keer)
            # 1. Het controleert of er stim-events zijn gevonden in de raw data.
            # 2. Als er geen stim-events zijn, wordt events_array_to_write op None gezet.
            # 3. Als er wel stim-events zijn, maar het aantal gevonden stim-events verschilt van het aantal gefilterde phase2-rows,
            #    wordt er een waarschuwing gegeven en worden de arrays afgestemd op de minimale lengte.
            if n_triggers == 0:
                print("  Geen stim-events gevonden in raw; geen events meegeven aan write_raw_bids.")
                events_array_to_write = None
            else:
                if n_triggers != n_filtered:
                    print(f"  Let op: aantal gevonden stim-events ({n_triggers}) ≠ aantal gefilterde phase2-rows ({n_filtered}).")
                    print("  Ik zal de arrays afstemmen op de minimale lengte (geen fouten raise).")
                n_use = min(n_triggers, n_filtered)
                if n_use == 0:
                    events_array_to_write = None
                else:
                    events_array_to_write = stim_events[:n_use].copy()
                    events_array_to_write[:, 2] = np.array(new_event_codes[:n_use], dtype=int)
                    print(f"  Events array klaargezet met {n_use} events (kolom 3 aangepast).")

        # BIDSPath maken (subject index i -> subjects[i])
        bids_path = BIDSPath(subject=subjects[i],
                             root=bids_dir,
                             datatype='eeg',
                             extension='.bdf',
                             suffix='eeg',
                             task='experiment')

        # Schrijf de events naar BIDS
        if events_array_to_write is None:   # als events ontbreken alleen bdf schrijven
            write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')
            # print(f"  {os.path.basename(bdf_file)} geschreven naar BIDS (zonder events).")
        else:
            write_raw_bids(raw,
                           bids_path=bids_path,
                           events_data=events_array_to_write,
                           event_id=event_id,
                           overwrite=True,
                           allow_preload=True,
                           format='EDF')
            # print(f"  {os.path.basename(bdf_file)} geschreven naar BIDS (met events).")
    print("\da files are in da BIDS, also dha blutoof dwife is ready to pairh...sorry ik heb mn koptelefoon net aangezet en het is laat")
    print("\nKlaar met alle bestanden in BIDS zetten.")

# run functie
rawtobids(raw_dir=raw_dir, bids_dir=bids_dir, subjects=subjects, event_id=event_id)

Gevonden .bdf bestanden: 3
Extracting EDF parameters from C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\raw\1_2_eeg.bdf...
BDF file detected
Setting channel info structure...


Creating raw.info structure...
Reading 0 ... 479231  =      0.000 ...   467.999 secs...
  Waarschuwing: events file niet gevonden: C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\raw\1_2_*_events.tsv. Schrijf BIDS zonder aangepaste events.
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\participants.tsv'...
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\participants.json'...
Writing 'C:/Users/Gebruiker/Desktop/Code Data Science/Code-Data-Science/bids/sub-01/eeg/sub-01_space-CapTrak_electrodes.tsv'...
Writing 'C:/Users/Gebruiker/Desktop/Code Data Science/Code-Data-Science/bids/sub-01/eeg/sub-01_space-CapTrak_coordsystem.json'...
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\dataset_description.json'...
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\sub-01\eeg\sub-01_task-experiment_eeg.json'...
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-D

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\137508163.py:24: RuntimeWarning: The unit for channel(s) EXG7, EXG8, M1, M2 has changed from V to NA.
  raw.set_channel_types({ch: 'misc' for ch in misc if ch in raw.ch_names})
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\137508163.py:92: RuntimeWarning: No events found or provided. Please add annotations to the raw data, or provide the events and event_id parameters. For resting state data, BIDS recommends naming the task using labels beginning with "rest".
  write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\137508163.py:92: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')


Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\sub-01\sub-01_scans.tsv'...
Wrote C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\sub-01\sub-01_scans.tsv entry with eeg\sub-01_task-experiment_eeg.edf.
Extracting EDF parameters from C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\raw\1_4_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 474111  =      0.000 ...   462.999 secs...
  Waarschuwing: events file niet gevonden: C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\raw\1_4_*_events.tsv. Schrijf BIDS zonder aangepaste events.
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\participants.tsv'...
Writing 'C:\Users\Gebruiker\Desktop\Code Data Science\Code-Data-Science\bids\participants.json'...
Writing 'C:/Users/Gebruiker/Desktop/Code Data Science/Code-Data-Science/bids/sub-02/eeg/sub-02_space-CapTrak_electrodes.tsv'...
Writi

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\137508163.py:24: RuntimeWarning: The unit for channel(s) EXG7, EXG8, M1, M2 has changed from V to NA.
  raw.set_channel_types({ch: 'misc' for ch in misc if ch in raw.ch_names})
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\137508163.py:92: RuntimeWarning: No events found or provided. Please add annotations to the raw data, or provide the events and event_id parameters. For resting state data, BIDS recommends naming the task using labels beginning with "rest".
  write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11552\137508163.py:92: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True, format='EDF')


👉 **Question:** Why would we bother with storing our data in BIDS format?

**Answer:**


## Part 2: Preprocess example data set

🔥 Please take an example block and load the eeg and events data (from the BIDS directory).

In [ ]:
# Your code goes here. Please add comments.
rawtobids(raw_dir)
# Sam

Gevonden bestanden: []


IndexError: list index out of range

🔥 Please plot the raw data. Show examples of (i) clean data, (ii) blinks (look at eog channels), and (iii) muscle artefacts. Print the shape of the data, and indicate what the dimensions correspond to. Print all channel names. Print all channel types.

In [ ]:
# Your code goes here. Please add comments.
# Irem

👉 **Question:** What kind of information can you extract just from visually inspecting raw EEG traces?

**Answer:**

🔥 Please set the channels 'M1', 'M2', 'EXG7' and 'EXG8' to type 'misc', and channels 'UP', 'DOWN', 'LEFT' and 'RIGH' to type 'eog'. Also set the right montage, and plot this. 

PS: Yes, it is annoying that we did this before. This information did end up correctly in the json sidecar files (check this), but not in the *eeg.bdf files.

In [ ]:
# Your code goes here. Please add comments.
# Irem

🔥 Please re-reference the EEG data to the average of the two mastoids.

In [ ]:
# Your code goes here. Please add comments.
# Roderik

👉 **Question:** Why would you want to re-reference the EEG data? What are common approaches? What are advantages and disadvantages of each?

**Answer:**

🔥 Please combine eog channels into bipolar horizontal ("HEOG") and vertical ("VEOG") EOG derivations. Check if the resulting channel type is correct.

In [ ]:
# Your code goes here. Please add comments.
# Roderik

👉 **Question:** Why do we take the difference between for example the UP and DOWN electrode?

**Answer:** 

🔥 Please apply a bandpass filter. Look up what a bandpass filter is, and use common cutoffs for EEG.

In [ ]:
# Your code goes here. Please add comments.
# Mernan

👉 **Question:** Why should we filter EEG data? What type of temporal filters are typically applied, and what are common cut-offs? 

**Answer:**

🔥 Please downsample the EEG data to 250Hz.

In [ ]:
# Your code goes here. Please add comments.
# Sam

👉 **Question:** What are the advantages of downsampling, and why does 250Hz makes sense?

**Answer:**

🔥 Please perform independent component analysis and plot the components.

In [ ]:
# Your code goes here. Please add comments.
# Sam

👉 **Question:** What does this plot show?  

**Answer:** 


🔥 Please remove components associated with blinks and eye movements. Include diagnostic figures (check with plotting methods exists for ica objects).

In [ ]:
# Your code goes here. Please add comments.
# Mernan

👉 **Question:** What do these plot show?  

**Answer:** 

🔥 Please plot the same bit of data, before and after ica-based artefact removal.

In [ ]:
# Your code goes here. Please add comments.
# Irem


👉 **Question:** What kinds of artifacts can ICA help to remove in EEG data? Why do we want to do this? 

**Answer:**

🔥 Please epoch the cleaned data (make sure to use "event_id"), by cutting out segments from 500ms before each sound to 1000ms after. Plot the resulting epochs object.

In [ ]:
# Your code goes here. Please add comments.
# Mernan


👉 **Question:** Why do we extract epochs relative to events?  

**Answer:**


## Part 3: Preprocess all the data

🔥 Please put all the code we wrote so far in function called ```preprocess_eeg_data```. The function should take "bids_path" as input and return "epochs", You can then loop across all data and concatenate all the epochs.

In [14]:
# Your code goes here. Please add comments.

## Part 4: Analyze the data

🔥 Please define the following sensor groupings: frontal, central, temporal, parietal and occipital.

In [70]:
# define sensor groupings:
frontal = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7',
           'Fp2', 'AF8', 'AF4', 'F2', 'F4', 'F6', 'F8']
central = ['FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'CP5', 'CP3', 'CP1',
           'FC6', 'FC4', 'FC2', 'C2', 'C4', 'C6', 'CP6', 'CP4', 'CP2']
temporal = ['FT7', 'T7', 'TP7', 'FT8', 'T8', 'TP8',]
parietal = ['P1', 'P3', 'P5', 'P7', 'P9', 
            'P2', 'P4', 'P6', 'P8', 'P10']
occipital = ['PO7', 'PO3', 'O1',
             'PO8', 'PO4', 'O2']